In [54]:
from __future__ import division
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
import xgboost as xgb
import lightgbm as lgb
import pickle
import time
import datetime
import math
import gc
import warnings
warnings.filterwarnings('ignore')

In [55]:
root_path = '../'  # '/media/xiaoxy/2018-Kaggle-AdTrackingFraud/'
predictors = []

In [56]:
########################################### Helper function ###########################################

In [57]:
def encode_onehot(df, column_name):
    df_onehot = pd.get_dummies(df[column_name], prefix=column_name)
    df_all = pd.concat([df.drop([column_name], axis=1), df_onehot], axis=1)
    predictors.append(column_name)
    return df_all


def encode_count(df, column_name):
    le = preprocessing.LabelEncoder()
    le.fit(list(df[column_name].values))
    df[column_name] = le.transform(list(df[column_name].values))
    predictors.append(column_name)
    return df


def merge_count(df, columns_groupby, new_column_name, type='uint64'):
    add = pd.DataFrame(df.groupby(columns_groupby).size()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_nunique(df, columns_groupby, column, new_column_name, type='uint64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].nunique()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_cumcount(df, columns_groupby, column, new_column_name, type='uint64'):
    df[new_column_name] = df.groupby(columns_groupby)[column].cumcount().values.astype(type)
    predictors.append(new_column_name)
    return df


def merge_median(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].median()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_mean(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].mean()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_sum(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].sum()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    # predictors.append(new_column_name)  # bug: twice
    return df


def merge_max(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].max()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_min(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].min()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_std(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].std()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_var(df, columns_groupby, column, new_column_name, type='float64'):
    add = pd.DataFrame(df.groupby(columns_groupby)[column].var()).reset_index()
    add.columns = columns_groupby + [new_column_name]
    df = df.merge(add, on=columns_groupby, how="left")
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_rank(df, columns_groupby, column, new_column_name, ascending=True, type='uint64'):
    df[new_column_name] = df.groupby(columns_groupby)[column].rank(ascending=ascending)
    df[new_column_name] = df[new_column_name].astype(type)
    predictors.append(new_column_name)
    return df


def merge_feat_count(df, df_feat, columns_groupby, column, new_column_name=""):
    df_count = pd.DataFrame(df_feat.groupby(columns_groupby)[column].count()).reset_index()
    if not new_column_name:
        df_count.columns = columns_groupby + [column + "_gb_%s_count" % ("_".join(columns_groupby))]
    else:
        df_count.columns = columns_groupby + [new_column_name]
    df = df.merge(df_count, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_count.columns[-1])
    return df


def merge_feat_nunique(df, df_feat, columns_groupby, column, new_column_name=""):
    df_nunique = pd.DataFrame(df_feat.groupby(columns_groupby)[column].nunique()).reset_index()
    if not new_column_name:
        df_nunique.columns = columns_groupby + [column + "_%s_nunique" % ("_".join(columns_groupby))]
    else:
        df_nunique.columns = columns_groupby + [new_column_name]
    df = df.merge(df_nunique, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_nunique.columns[-1])
    return df


def merge_feat_mean(df, df_feat, columns_groupby, column, new_column_name=""):
    df_mean = pd.DataFrame(df_feat.groupby(columns_groupby)[column].mean()).reset_index()
    if not new_column_name:
        df_mean.columns = columns_groupby + [column + "_%s_mean" % ("_".join(columns_groupby))]
    else:
        df_mean.columns = columns_groupby + [new_column_name]
    df = df.merge(df_mean, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_mean.columns[-1])
    return df


def merge_feat_std(df, df_feat, columns_groupby, column, new_column_name=""):
    df_std = pd.DataFrame(df_feat.groupby(columns_groupby)[column].std()).reset_index()
    if not new_column_name:
        df_std.columns = columns_groupby + [column + "_%s_std" % ("_".join(columns_groupby))]
    else:
        df_std.columns = columns_groupby + [new_column_name]
    df = df.merge(df_std, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_std.columns[-1])
    return df


def merge_feat_median(df, df_feat, columns_groupby, column, new_column_name=""):
    df_median = pd.DataFrame(df_feat.groupby(columns_groupby)[column].median()).reset_index()
    if not new_column_name:
        df_median.columns = columns_groupby + [column + "_%s_median" % ("_".join(columns_groupby))]
    else:
        df_median.columns = columns_groupby + [new_column_name]
    df = df.merge(df_median, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_median.columns[-1])
    return df


def merge_feat_max(df, df_feat, columns_groupby, column, new_column_name=""):
    df_max = pd.DataFrame(df_feat.groupby(columns_groupby)[column].max()).reset_index()
    if not new_column_name:
        df_max.columns = columns_groupby + [column + "_%s_max" % ("_".join(columns_groupby))]
    else:
        df_max.columns = columns_groupby + [new_column_name]
    df = df.merge(df_max, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_max.columns[-1])
    return df


def merge_feat_min(df, df_feat, columns_groupby, column, new_column_name=""):
    df_min = pd.DataFrame(df_feat.groupby(columns_groupby)[column].min()).reset_index()
    if not new_column_name:
        df_min.columns = columns_groupby + [column + "_%s_min" % ("_".join(columns_groupby))]
    else:
        df_min.columns = columns_groupby + [new_column_name]
    df = df.merge(df_min, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_min.columns[-1])
    return df


def merge_feat_sum(df, df_feat, columns_groupby, column, new_column_name=""):
    df_sum = pd.DataFrame(df_feat.groupby(columns_groupby)[column].sum()).reset_index()
    if not new_column_name:
        df_sum.columns = columns_groupby + [column + "_%s_sum" % ("_".join(columns_groupby))]
    else:
        df_sum.columns = columns_groupby + [new_column_name]
    df = df.merge(df_sum, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_sum.columns[-1])
    return df


def merge_feat_var(df, df_feat, columns_groupby, column, new_column_name=""):
    df_var = pd.DataFrame(df_feat.groupby(columns_groupby)[column].var()).reset_index()
    if not new_column_name:
        df_var.columns = columns_groupby + [column + "_%s_var" % ("_".join(columns_groupby))]
    else:
        df_var.columns = columns_groupby + [new_column_name]
    df = df.merge(df_var, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_var.columns[-1])
    return df


def merge_feat_quantile(df, df_feat, columns_groupby, column, quantile_n, new_column_name=""):
    df_quantile = pd.DataFrame(df_feat.groupby(columns_groupby)[column].quantile(quantile_n)).reset_index()
    if not new_column_name:
        df_quantile.columns = columns_groupby + [column + "_%s_quantile" % ("_".join(columns_groupby))]
    else:
        df_quantile.columns = columns_groupby + [new_column_name]
    df = df.merge(df_quantile, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_quantile.columns[-1])
    return df


def merge_feat_skew(df, df_feat, columns_groupby, column, new_column_name=""):
    df_skew = pd.DataFrame(df_feat.groupby(columns_groupby)[column].skew()).reset_index()
    if not new_column_name:
        df_skew.columns = columns_groupby + [column + "_%s_skew" % ("_".join(columns_groupby))]
    else:
        df_skew.columns = columns_groupby + [new_column_name]
    df = df.merge(df_skew, on=columns_groupby, how="left").fillna(0)
    predictors.append(df_skew.columns[-1])
    return df


def merge_rank_sp(df, feat1, feat2, ascending):
    df.sort_values([feat1, feat2], inplace=True, ascending=ascending)
    df['rank'] = range(df.shape[0])
    min_rank = df.groupby(feat1, as_index=False)['rank'].agg({'min_rank': 'min'})
    df = pd.merge(df, min_rank, on=feat1, how='left')
    df['rank'] = df['rank'] - df['min_rank']
    predictors.append('rank')
    del df['min_rank']
    return df


def log(info):
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' ' + str(info))


def log_shape(train, test):
    log('Train data shape: %s' % str(train.shape))
    log('Test data shape: %s' % str(test.shape))

In [58]:
def process_date(df):
    format = '%Y-%m-%d %H:%M:%S'
    df['date'] = pd.to_datetime(df['click_time'], format=format)
    df['month'] = df['date'].dt.month.astype('uint8')
    df['weekday'] = df['date'].dt.weekday.astype('uint8')
    df['day'] = df['date'].dt.day.astype('uint8')
    df['hour'] = df['date'].dt.hour.astype('uint8')
    df['minute'] = df['date'].dt.minute.astype('uint8')
    df['second'] = df['date'].dt.second.astype('uint8')
    df['tm_hour'] = (df['hour'] + df['minute'] / 60.0).astype('float32')
    df['tm_hour_sin'] = (df['tm_hour'].map(lambda x: math.sin((x - 12) / 24 * 2 * math.pi))).astype('float32')
    df['tm_hour_cos'] = (df['tm_hour'].map(lambda x: math.cos((x - 12) / 24 * 2 * math.pi))).astype('float32')
    del df['click_time']
    return df

In [59]:
########### Construct features function - begin ###########

In [60]:
def cal_next_time_delta(df, suffix, type='float32'):
    groupby_columns = [
        {'columns': ['ip', 'app', 'channel', 'device', 'os']},
        {'columns': ['ip', 'os', 'device']},
        {'columns': ['ip', 'os', 'device', 'app']}
    ]
    # Calculate the time to next click for each group
    for spec in groupby_columns:
        # Name of new feature
        new_name = '{}_{}'.format('_'.join(spec['columns']), suffix)
        # Unique list of features to select
        all_features = spec['columns'] + ['date']
        # Run calculation
        log('Calculate ' + suffix + '...')
        df[new_name] = (df[all_features].groupby(spec['columns']).date.shift(-1) - df.date).dt.seconds.astype(type)
        predictors.append(new_name)
        gc.collect()
    return df


def cal_prev_time_delta(df, suffix, type='float32'):
    groupby_columns = [
        {'columns': ['ip', 'channel']},
        {'columns': ['ip', 'os']}
    ]
    # Calculate the time to prev click for each group
    for spec in groupby_columns:
        # Name of new feature
        new_name = '{}_{}'.format('_'.join(spec['columns']), suffix)
        # Unique list of features to select
        all_features = spec['columns'] + ['date']
        # Run calculation
        log('Calculate ' + suffix + '...')
        df[new_name] = (df.date - df[all_features].groupby(spec['columns']).date.shift(+1)).dt.seconds.astype(type)
        predictors.append(new_name)
        gc.collect()
    return df


def cal_cvr(train, test, type='float32'):
    train['cvr_gb_ip_day_hour'] = 0
    train['cvr_gb_ip_app'] = 0
    train['cvr_gb_ip_app_os'] = 0

    # Define group by list
    idh = ['ip', 'day', 'hour']
    ia = ['ip', 'app']
    iao = ['ip', 'app', 'os']

#     kf = KFold(train.shape[0], n_splits=5, shuffle=True, random_state=7)
    kf = KFold(n_splits=5, shuffle=True, random_state=7)

    for i, (train_index, test_index) in enumerate(kf.split(train)):
        log('Fold ' + str(i) + ' begin...')

        # Divide train/test fold
        tr = train.iloc[train_index, :train.shape[1] - 3]
        te = train.iloc[test_index, :train.shape[1] - 3]

        # Calculate sum of label of train folds
        log('Cal sum_label_gb_ip_day_hour')
        tr = merge_sum(tr, idh, 'is_attributed', 'sum_label_gb_ip_day_hour')
        log('Cal sum_label_gb_ip_app')
        tr = merge_sum(tr, ia, 'is_attributed', 'sum_label_gb_ip_app')
        log('Cal sum_label_gb_ip_app_os')
        tr = merge_sum(tr, iao, 'is_attributed', 'sum_label_gb_ip_app_os')

        # Calculate cvr of train folds with using smothing technique
        tr['cvr_gb_ip_day_hour'] = GaussianSmoth().update_moment(tr['count_gb_ip_day_hour'], tr['sum_label_gb_ip_day_hour'])
        tr['cvr_gb_ip_app'] = GaussianSmoth().update_moment(tr['count_gb_ip_app'], tr['sum_label_gb_ip_app'])
        tr['cvr_gb_ip_app_os'] = GaussianSmoth().update_moment(tr['count_gb_ip_app_os'], tr['sum_label_gb_ip_app_os'])

        # Merge test fold with cvr features of train folds
        te = te.merge(tr[['cvr_gb_ip_day_hour'] + idh].drop_duplicates(subset=idh, keep='first'), on=idh, how='left')
        te = te.merge(tr[['cvr_gb_ip_app'] + ia].drop_duplicates(subset=ia, keep='first'), on=ia, how='left')
        te = te.merge(tr[['cvr_gb_ip_app_os'] + iao].drop_duplicates(subset=iao, keep='first'), on=iao, how='left')

        # Put it in train
        train['cvr_gb_ip_day_hour'] += te['cvr_gb_ip_day_hour']
        train['cvr_gb_ip_app'] += te['cvr_gb_ip_app']
        train['cvr_gb_ip_app_os'] += te['cvr_gb_ip_app_os']

        del tr, te
        log('Fold ' + str(i) + ' Done!')

    # Convert type
    train['cvr_gb_ip_day_hour'] = train['cvr_gb_ip_day_hour'].astype(type)
    train['cvr_gb_ip_app'] = train['cvr_gb_ip_app'].astype(type)
    train['cvr_gb_ip_app_os'] = train['cvr_gb_ip_app_os'].astype(type)

    # Merge cvr of train to test
    test = test.merge(train[['cvr_gb_ip_day_hour'] + idh].drop_duplicates(subset=idh, keep='first'), on=idh, how='left')
    test = test.merge(train[['cvr_gb_ip_app'] + ia].drop_duplicates(subset=ia, keep='first'), on=ia, how='left')
    test = test.merge(train[['cvr_gb_ip_app_os'] + iao].drop_duplicates(subset=iao, keep='first'), on=iao, how='left')

    predictors.append('cvr_gb_ip_day_hour')
    predictors.append('cvr_gb_ip_app')
    predictors.append('cvr_gb_ip_app_os')

    return train, test

In [61]:
########### Construct features function - end ###########

In [62]:
def spilt_local_train_test(df, train_size, test_size):
    local_train = df[:train_size]
    local_test = df[train_size:train_size + test_size]
    return local_train, local_test


def get_model_input_data(train, test, is_local):
    feat = ['ip', 'app', 'device', 'os', 'channel', 'hour']
    for f in feat:
        if f not in predictors:
            predictors.append(f)
    train_x = train[predictors]
    train_y = train.is_attributed.values
    if is_local == 1:
        test_x = test[train_x.columns.values]
        test_y = test.is_attributed.values
        return train_x, train_y, test_x, test_y
    else:
        test_x = test[train_x.columns.values]
        return train_x, train_y, test_x

In [63]:
def xgb_cv(train_feature, train_label, test_feature, test_label, params, folds, rounds):
    start = time.clock()
    print(train_feature.columns)
    params['scale_pos_weights'] = float(len(train_label[train_label == 0])) / len(train_label[train_label == 1])
    dtrain = xgb.DMatrix(train_feature, label=train_label)
    dtest = xgb.DMatrix(test_feature, label=test_label)
    num_round = rounds
    watchlist = [(dtrain, 'train'), (dtest, 'test')]
    print('XGBoost run cv: ' + 'round: ' + str(rounds))
    res = xgb.train(params, dtrain, num_round, watchlist, verbose_eval=20, early_stopping_rounds=50)
    elapsed = (time.clock() - start)
    print('Time used:' + str(elapsed) + 's')
    return res.best_ntree_limit, res.best_score, res


def xgb_predict(train_feature, train_label, test_feature, rounds, params):
    params['scale_pos_weights'] = float(len(train_label[train_label == 0])) / len(train_label[train_label == 1])
    dtrain = xgb.DMatrix(train_feature, label=train_label)
    dtest = xgb.DMatrix(test_feature, label=np.zeros(test_feature.shape[0]))
    watchlist = [(dtrain, 'train')]
    num_round = rounds
    model = xgb.train(params, dtrain, num_round, watchlist, verbose_eval=30)
    predict = model.predict(dtest)
    return model, predict

In [64]:
def lgb_cv(train_feature, train_label, test_feature, test_label, params, folds, rounds):
    start = time.clock()
    print(train_feature.columns)
    params['scale_pos_weight'] = float(len(train_label[train_label == 0])) / len(train_label[train_label == 1])
    dtrain = lgb.Dataset(train_feature, label=train_label, categorical_feature=['app', 'device', 'os', 'channel', 'hour'])
    dtest = lgb.Dataset(test_feature, label=test_label, categorical_feature=['app', 'device', 'os', 'channel', 'hour'])
    num_round = rounds
    print('LightGBM run cv: ' + 'round: ' + str(rounds))
    res = lgb.train(params, dtrain, num_round, valid_sets=[dtest], valid_names=['test'], verbose_eval=1, early_stopping_rounds=20)
    elapsed = (time.clock() - start)
    print('Time used:', elapsed, 's')
    return res.best_iteration, res.best_score['test']['auc'], res


def lgb_predict(train_feature, train_label, test_feature, rounds, params):
    dtrain = lgb.Dataset(train_feature, label=train_label, categorical_feature=['app', 'device', 'os', 'channel', 'hour'])
    num_round = rounds
    model = lgb.train(params, dtrain, num_round, valid_sets=[dtrain], verbose_eval=1)
    predict = model.predict(test_feature)
    return model, predict

In [65]:
def store_result(test_index, pred, name):
    result = pd.DataFrame({'click_id': test_index, 'is_attributed': pred})
    result.to_csv(root_path + 'data/output/sub/' + name + '.csv', index=False, sep=',')
    return result

In [66]:
class GaussianSmoth(object):
    def __init__(self, alpha=0, beta=0):
        self.alpha = alpha
        self.beta = beta

    def update_moment(self, tries, success):
        '''estimate alpha, beta using moment estimation'''
        mean, var = self.__compute_moment(tries, success)
        self.alpha = (mean + 0.000001) * ((mean + 0.000001) * (1.000001 - mean) / (var + 0.000001) - 1)
        self.beta = (1.000001 - mean) * ((mean + 0.000001) * (1.000001 - mean) / (var + 0.000001) - 1)
        print(self.alpha, self.beta)
        return (self.alpha + success) / (self.alpha + self.beta + tries)

    def __compute_moment(self, tries, success):
        # Cal mean and variance
        '''moment estimation'''
        ctr_list = []
        mean = (success / tries).mean()
        if len(tries) == 1:
            var = 0
        else:
            var = (success / tries).var()
        return mean, var

In [67]:
########################################### Read data ###########################################

In [68]:
log('Read data...')
dtypes = {
    'click_id': 'uint32',
    'ip': 'uint32',
    'app': 'uint16',
    'device': 'uint16',
    'os': 'uint16',
    'channel': 'uint16',
    'is_attributed': 'uint8'
}
train = pd.read_csv( 'train.csv', header=0, sep=',', dtype=dtypes, usecols=['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed'],nrows=1000000 )
test_supplement = pd.read_csv( 'test_supplement.csv', header=0, sep=',', dtype=dtypes, usecols=['ip', 'app', 'device', 'os', 'channel', 'click_time'], nrows=1000000)
gc.collect()
log('Read data done!')
log_shape(train, test_supplement)

2019-04-23 17:32:27 Read data...
2019-04-23 17:32:29 Read data done!
2019-04-23 17:32:29 Train data shape: (1000000, 7)
2019-04-23 17:32:29 Test data shape: (1000000, 6)


In [69]:
########################################### Preprocess ###########################################

In [70]:
log('Process date...')
train = process_date(train)
test_supplement = process_date(test_supplement)
gc.collect()
log('Process date done!')
log_shape(train, test_supplement)

2019-04-23 17:32:29 Process date...
2019-04-23 17:32:35 Process date done!
2019-04-23 17:32:35 Train data shape: (1000000, 16)
2019-04-23 17:32:35 Test data shape: (1000000, 15)


In [71]:
########################################### Feature engineer ###########################################

In [72]:
train_len = len(train)
log('Train size:' + str(train_len))

2019-04-23 17:32:35 Train size:1000000


In [73]:
log('Train append test_supplement...')
df = train.append(test_supplement).reset_index(drop=True)
del train
del test_supplement
gc.collect()
log('Train append test_supplement done!')

2019-04-23 17:32:35 Train append test_supplement...
2019-04-23 17:32:36 Train append test_supplement done!


In [74]:
log('Before feature engineer')
log('Num of features: ' + str(len(df.columns)))
log('Features: ' + str(df.columns))

2019-04-23 17:32:36 Before feature engineer
2019-04-23 17:32:36 Num of features: 16
2019-04-23 17:32:36 Features: Index(['app', 'channel', 'date', 'day', 'device', 'hour', 'ip',
       'is_attributed', 'minute', 'month', 'os', 'second', 'tm_hour',
       'tm_hour_cos', 'tm_hour_sin', 'weekday'],
      dtype='object')


In [75]:
# Construct features...

In [76]:
log('Cal next_time_delta')
df = cal_next_time_delta(df, 'next_time_delta', 'float32')
gc.collect()
log('Cal prev_time_delta')
df = cal_prev_time_delta(df, 'prev_time_delta', 'float32')
gc.collect()

2019-04-23 17:32:36 Cal next_time_delta
2019-04-23 17:32:36 Calculate next_time_delta...
2019-04-23 17:32:37 Calculate next_time_delta...
2019-04-23 17:32:38 Calculate next_time_delta...
2019-04-23 17:32:40 Cal prev_time_delta
2019-04-23 17:32:40 Calculate prev_time_delta...
2019-04-23 17:32:41 Calculate prev_time_delta...


0

In [77]:
log('Cal nunique_channel_gb_ip')
df = merge_nunique(df, ['ip'], 'channel', 'nunique_channel_gb_ip', 'uint32')
gc.collect()
log('Cal nunique_app_gb_ip_device_os')
df = merge_nunique(df, ['ip', 'device', 'os'], 'app', 'nunique_app_gb_ip_device_os', 'uint32')
gc.collect()
log('Cal nunique_hour_gb_ip_day')
df = merge_nunique(df, ['ip', 'day'], 'hour', 'nunique_hour_gb_ip_day', 'uint32')
gc.collect()
log('Cal nunique_app_gb_ip')
df = merge_nunique(df, ['ip'], 'app', 'nunique_app_gb_ip', 'uint32')
gc.collect()
log('Cal nunique_os_gb_ip_app')
df = merge_nunique(df, ['ip', 'app'], 'os', 'nunique_os_gb_ip_app', 'uint32')
gc.collect()
log('Cal nunique_device_gb_ip')
df = merge_nunique(df, ['ip'], 'device', 'nunique_device_gb_ip', 'uint32')
gc.collect()
log('Cal nunique_channel_gb_app')
df = merge_nunique(df, ['app'], 'channel', 'nunique_channel_gb_app', 'uint32')
gc.collect()

2019-04-23 17:32:42 Cal nunique_channel_gb_ip
2019-04-23 17:32:45 Cal nunique_app_gb_ip_device_os
2019-04-23 17:32:50 Cal nunique_hour_gb_ip_day
2019-04-23 17:32:53 Cal nunique_app_gb_ip
2019-04-23 17:32:56 Cal nunique_os_gb_ip_app
2019-04-23 17:33:01 Cal nunique_device_gb_ip
2019-04-23 17:33:04 Cal nunique_channel_gb_app


42

In [78]:
log('Cal cumcount_os_gb_ip')
df = merge_cumcount(df, ['ip'], 'os', 'cumcount_os_gb_ip', 'uint32');
gc.collect()
log('Cal cumcount_app_gb_ip_device_os')
df = merge_cumcount(df, ['ip', 'device', 'os'], 'app', 'cumcount_app_gb_ip_device_os', 'uint32');
gc.collect()

2019-04-23 17:33:07 Cal cumcount_os_gb_ip
2019-04-23 17:33:08 Cal cumcount_app_gb_ip_device_os


7

In [79]:
log('Cal count_gb_ip_day_hour')
df = merge_count(df, ['ip', 'day', 'hour'], 'count_gb_ip_day_hour', 'uint32');
gc.collect()
log('Cal count_gb_ip_app')
df = merge_count(df, ['ip', 'app'], 'count_gb_ip_app', 'uint32');
gc.collect()
log('Cal count_gb_ip_app_os')
df = merge_count(df, ['ip', 'app', 'os'], 'count_gb_ip_app_os', 'uint32');
gc.collect()

2019-04-23 17:33:09 Cal count_gb_ip_day_hour
2019-04-23 17:33:11 Cal count_gb_ip_app
2019-04-23 17:33:14 Cal count_gb_ip_app_os


49

In [80]:
log('Cal var_day_gb_ip_app_os')
df = merge_var(df, ['ip', 'app', 'os'], 'day', 'var_day_gb_ip_app_os', 'float32')
gc.collect()

2019-04-23 17:33:18 Cal var_day_gb_ip_app_os


49

In [81]:
# Construct features done!

In [82]:
log('After feature engineer')
log('Num of features: ' + str(len(df.columns)))
log('Features: ' + str(df.columns))

2019-04-23 17:33:22 After feature engineer
2019-04-23 17:33:22 Num of features: 34
2019-04-23 17:33:22 Features: Index(['app', 'channel', 'date', 'day', 'device', 'hour', 'ip',
       'is_attributed', 'minute', 'month', 'os', 'second', 'tm_hour',
       'tm_hour_cos', 'tm_hour_sin', 'weekday',
       'ip_app_channel_device_os_next_time_delta',
       'ip_os_device_next_time_delta', 'ip_os_device_app_next_time_delta',
       'ip_channel_prev_time_delta', 'ip_os_prev_time_delta',
       'nunique_channel_gb_ip', 'nunique_app_gb_ip_device_os',
       'nunique_hour_gb_ip_day', 'nunique_app_gb_ip', 'nunique_os_gb_ip_app',
       'nunique_device_gb_ip', 'nunique_channel_gb_app', 'cumcount_os_gb_ip',
       'cumcount_app_gb_ip_device_os', 'count_gb_ip_day_hour',
       'count_gb_ip_app', 'count_gb_ip_app_os', 'var_day_gb_ip_app_os'],
      dtype='object')


In [83]:
########### All features save & reload - begin ###########

In [84]:
# Save all features
pickle.dump(df, open('all.p', 'wb'))

In [85]:
# # Reload all features
# df = cPickle.load(open(root_path + 'data/output/feat/all.p', 'rb'))
# train_len = 184903891
# dtypes = {
#     'click_id': 'uint32',
#     'ip': 'uint32',
#     'app': 'uint16',
#     'device': 'uint16',
#     'os': 'uint16',
#     'channel': 'uint16',
#     'is_attributed': 'uint8'
# }
# predictors = ['ip', 'app', 'device', 'os', 'channel', 'hour',
#               'next_time_delta', 'prev_time_delta',
#               'nunique_channel_gb_ip', 'nunique_app_gb_ip_device_os',
#               'nunique_hour_gb_ip_day', 'nunique_app_gb_ip', 'nunique_os_gb_ip_app',
#               'nunique_device_gb_ip', 'nunique_channel_gb_app',
#               'cumcount_os_gb_ip', 'cumcount_app_gb_ip_device_os',
#               'count_gb_ip_day_hour', 'count_gb_ip_app', 'count_gb_ip_app_os',
#               'var_day_gb_ip_app_os']

In [86]:
########### All features save & reload - end ###########

In [87]:
log('Train test_supplement divid...')
train = df[:train_len]
test_supplement = df[train_len:]
del df
gc.collect()
log_shape(train, test_supplement)
log('Train test_supplement divid done!')

2019-04-23 17:33:25 Train test_supplement divid...
2019-04-23 17:33:25 Train data shape: (1000000, 34)
2019-04-23 17:33:25 Test data shape: (1000000, 34)
2019-04-23 17:33:25 Train test_supplement divid done!


In [88]:
log('Read test...')
test = pd.read_csv('test.csv', header=0, sep=',', dtype=dtypes, usecols=['click_id', 'ip', 'app', 'device', 'os', 'channel', 'click_time'], parse_dates=['click_time'],nrows=1000000)
log('Test data original shape: ' + str(test.shape))

2019-04-23 17:33:25 Read test...
2019-04-23 17:33:27 Test data original shape: (1000000, 7)


In [89]:
test = test.merge(test_supplement.drop_duplicates(subset=['ip', 'app', 'device', 'os', 'channel', 'date'], keep='first'), left_on=['ip', 'app', 'device', 'os', 'channel', 'click_time'], right_on=['ip', 'app', 'device', 'os', 'channel', 'date'], how='left')
test.drop(['click_time'], axis=1, inplace=True)
del test_supplement
gc.collect()
log_shape(train, test)
log('Read test done!')

2019-04-23 17:33:30 Train data shape: (1000000, 34)
2019-04-23 17:33:30 Test data shape: (1000000, 35)
2019-04-23 17:33:30 Read test done!


In [90]:
# Cal cvr features
log('Cal cvr...')
train, test = cal_cvr(train, test, 'float32')
log('Cal cvr done!')

2019-04-23 17:33:30 Cal cvr...
2019-04-23 17:33:30 Fold 0 begin...
2019-04-23 17:33:30 Cal sum_label_gb_ip_day_hour
2019-04-23 17:33:31 Cal sum_label_gb_ip_app
2019-04-23 17:33:33 Cal sum_label_gb_ip_app_os
0.002138335292281242 1.4507306562129352
0.00037441349172921966 0.24676074136682433
0.00031851350592673847 0.2007977547378011
2019-04-23 17:33:36 Fold 0 Done!
2019-04-23 17:33:36 Fold 1 begin...
2019-04-23 17:33:37 Cal sum_label_gb_ip_day_hour
2019-04-23 17:33:38 Cal sum_label_gb_ip_app
2019-04-23 17:33:39 Cal sum_label_gb_ip_app_os
0.0021517325982386756 1.4596803911856793
0.0003722526276228888 0.24556140552068823
0.0003029650914739995 0.19142063608413423
2019-04-23 17:33:42 Fold 1 Done!
2019-04-23 17:33:42 Fold 2 begin...
2019-04-23 17:33:43 Cal sum_label_gb_ip_day_hour
2019-04-23 17:33:44 Cal sum_label_gb_ip_app
2019-04-23 17:33:45 Cal sum_label_gb_ip_app_os
0.0021804954149106832 1.4632952845067138
0.0003845161579382355 0.25030797183774706
0.00031212443826963537 0.19458985730470296

In [91]:
########### CVR features save & reload - begin ###########

In [92]:
cvr_feats = ['cvr_gb_ip_day_hour', 'cvr_gb_ip_app', 'cvr_gb_ip_app_os']

In [93]:
# Save cvr features
pickle.dump(train[cvr_feats], open('train_cvr.p', 'wb'))
pickle.dump(test[cvr_feats], open('test_cvr.p', 'wb'))

In [94]:
# # Reload cvr features
# train_cvr = cPickle.load(open(root_path + 'data/output/feat/train_cvr.p', 'rb'))
# test_cvr = cPickle.load(open(root_path + 'data/output/feat/test_cvr.p', 'rb'))
# train = pd.concat([train, train_cvr], axis=1)
# test_cvr = pd.concat([test, test_cvr], axis=1)
# del train_cvr, test_cvr

In [95]:
########### CVR features save & reload - end ###########

In [96]:
########################################### Split dataset for local ###########################################

In [97]:
log('Split dataset to get local train/test set...')
local_train_size = 750000  # 182403890
local_test_size = 250000
local_train, local_test = spilt_local_train_test(train, local_train_size, local_test_size)
log('Split dataset to get local train/test set done!')

log('================================= Local data info =====================================')
log('Local train shape:' + str(local_train.shape))
log('Local test shape:' + str(local_test.shape))
log('Local train label ratio (0-1):' + str(local_train.is_attributed.value_counts().values * 1.0 / local_train.shape[0]))
log('Local train label number (0-1):' + str(local_train.is_attributed.value_counts().values))
log('Local train min/max date:' + str(local_train.date.min()) + ',' + str(local_train.date.max()))
log('Local test min/max date:' + str(local_test.date.min()) + ',' + str(local_test.date.max()))
log('=======================================================================================')

log('================================= Online data info =====================================')
log('Online train shape:' + str(train.shape))
log('Online test shape:' + str(test.shape))
log('Online train label ratio (0-1):' + str(train.is_attributed.value_counts().values * 1.0 / train.shape[0]))
log('Online train label number (0-1):' + str(train.is_attributed.value_counts().values))
log('Online train min/max date:' + str(train.date.min()) + ',' + str(train.date.max()))
log('Online train min/max date:' + str(test.date.min()) + ',' + str(test.date.max()))
log('=======================================================================================')

2019-04-23 17:34:05 Split dataset to get local train/test set...
2019-04-23 17:34:05 Split dataset to get local train/test set done!
2019-04-23 17:34:05 ================================= Local data info =====================================
2019-04-23 17:34:05 Local train shape:(750000, 37)
2019-04-23 17:34:05 Local test shape:(250000, 37)
2019-04-23 17:34:05 Local train label ratio (0-1):[0.99827333 0.00172667]
2019-04-23 17:34:05 Local train label number (0-1):[748705   1295]
2019-04-23 17:34:05 Local train min/max date:2017-11-06 14:32:21,2017-11-06 16:15:49
2019-04-23 17:34:05 Local test min/max date:2017-11-06 16:15:49,2017-11-06 16:21:51
2019-04-23 17:34:05 =======================================================================================
2019-04-23 17:34:05 ================================= Online data info =====================================
2019-04-23 17:34:05 Online train shape:(1000000, 37)
2019-04-23 17:34:05 Online test shape:(1000000, 38)
2019-04-23 17:34:05 Online

In [98]:
log('Get local model input data...')
local_train_x, local_train_y, local_test_x, local_test_y = get_model_input_data(local_train, local_test, is_local=1)
del local_train
del local_test
gc.collect()
log_shape(local_train_x, local_test_x)
log('Get local model input data done!')

2019-04-23 17:34:05 Get local model input data...
2019-04-23 17:34:05 Train data shape: (750000, 27)
2019-04-23 17:34:05 Test data shape: (250000, 27)
2019-04-23 17:34:05 Get local model input data done!


In [99]:
log('Get online model input data...')
online_train_x, online_train_y, online_test_x = get_model_input_data(train, test, is_local=0)
del train
del test
gc.collect()
log_shape(online_train_x, online_test_x)
log('Get online model input data done!')

2019-04-23 17:34:05 Get online model input data...
2019-04-23 17:34:06 Train data shape: (1000000, 27)
2019-04-23 17:34:06 Test data shape: (1000000, 27)
2019-04-23 17:34:06 Get online model input data done!


In [100]:
########################################### LigthGBM ###########################################

In [101]:
config_lgb = {
    'rounds': 100,
    'folds': 5
}

params_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'xentropy',
    'metric': 'auc',
    'learning_rate': 0.02,
    # 'is_unbalance': 'true',  # Because training data is unbalance (replaced with scale_pos_weight)
    'scale_pos_weight': 200,  # Because training data is extremely unbalanced
    'num_leaves': 31,  # We should let it be smaller than 2^(max_depth)
    'max_depth': -1,  # -1 means no limit
    'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
    'max_bin': 128,  # Number of bucketed bin for feature values
    'subsample': 0.7,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # Frequence of subsample, <=0 means no enable
    'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
    'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    'subsample_for_bin': 200000,  # Number of samples for constructing bin
    'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
    'reg_alpha': 0.99,  # L1 regularization term on weights
    'reg_lambda': 0.9,  # L2 regularization term on weights
    'nthread': 24,
    'verbose': 1,
    'seed': 8
}

In [102]:
local_test_x.shape

(250000, 27)

In [103]:
iterations_lgb, best_score_lgb, model_cv_lgb = lgb_cv(local_train_x, local_train_y, local_test_x, local_test_y, params_lgb, config_lgb['folds'], config_lgb['rounds'])

Index(['ip_app_channel_device_os_next_time_delta',
       'ip_os_device_next_time_delta', 'ip_os_device_app_next_time_delta',
       'ip_channel_prev_time_delta', 'ip_os_prev_time_delta',
       'nunique_channel_gb_ip', 'nunique_app_gb_ip_device_os',
       'nunique_hour_gb_ip_day', 'nunique_app_gb_ip', 'nunique_os_gb_ip_app',
       'nunique_device_gb_ip', 'nunique_channel_gb_app', 'cumcount_os_gb_ip',
       'cumcount_app_gb_ip_device_os', 'count_gb_ip_day_hour',
       'count_gb_ip_app', 'count_gb_ip_app_os', 'var_day_gb_ip_app_os',
       'cvr_gb_ip_day_hour', 'cvr_gb_ip_app', 'cvr_gb_ip_app_os', 'ip', 'app',
       'device', 'os', 'channel', 'hour'],
      dtype='object')
LightGBM run cv: round: 100
[1]	test's auc: 0.956363
Training until validation scores don't improve for 20 rounds.
[2]	test's auc: 0.958833
[3]	test's auc: 0.960584
[4]	test's auc: 0.960882
[5]	test's auc: 0.96105
[6]	test's auc: 0.961396
[7]	test's auc: 0.961133
[8]	test's auc: 0.961427
[9]	test's auc: 0.961505


In [104]:
pred_lgb = model_cv_lgb.predict(online_test_x)

In [105]:
model_lgb, pred_lgb = lgb_predict(online_train_x, online_train_y, online_test_x, iterations_lgb, params_lgb)

[1]	training's auc: 0.94116
[2]	training's auc: 0.957142
[3]	training's auc: 0.954473
[4]	training's auc: 0.956927
[5]	training's auc: 0.960503
[6]	training's auc: 0.960881
[7]	training's auc: 0.961142
[8]	training's auc: 0.961868
[9]	training's auc: 0.962767
[10]	training's auc: 0.962882
[11]	training's auc: 0.964428
[12]	training's auc: 0.965144
[13]	training's auc: 0.965172
[14]	training's auc: 0.965256
[15]	training's auc: 0.965376
[16]	training's auc: 0.965432
[17]	training's auc: 0.965752
[18]	training's auc: 0.965798
[19]	training's auc: 0.965698
[20]	training's auc: 0.966001
[21]	training's auc: 0.96599
[22]	training's auc: 0.965941
[23]	training's auc: 0.965949
[24]	training's auc: 0.966054
[25]	training's auc: 0.966583
[26]	training's auc: 0.966642
[27]	training's auc: 0.966787
[28]	training's auc: 0.96675
[29]	training's auc: 0.966945
[30]	training's auc: 0.966966
[31]	training's auc: 0.967203
[32]	training's auc: 0.967146
[33]	training's auc: 0.96721
[34]	training's auc: 0.

In [107]:
importance_lgb = sorted(zip(online_train_x.columns, model_cv_lgb.feature_importance("gain")), key=lambda x: x[1], reverse=True)
importance_lgb = pd.DataFrame({'feature': importance_lgb})
importance_lgb = importance_lgb.apply(lambda x: pd.Series(x['feature']), axis=1)
importance_lgb.columns = ['feature', 'importance']
importance_lgb.to_csv('importance-lgb-20180507-%f(r%d).csv' % (best_score_lgb, iterations_lgb), index=False)

In [109]:
res_lgb = store_result(pd.read_csv('test.csv', header=0, sep=',', usecols=['click_id']).click_id.astype(int), pred_lgb, '20180507-lgb-%f(r%d)' % (best_score_lgb, iterations_lgb))

ValueError: array length 1000000 does not match index length 18790469

In [ ]:
########### Model save and reload - begin ###########

In [ ]:
# Save model
log('Save model...')
model_lgb.save_model(root_path + 'data/output/model/lgb-%f(r%d).txt' % (best_score_lgb, iterations_lgb))
log('Model best score:' + str(best_score_lgb))
log('Model best iteration:' + str(iterations_lgb))
log('Save model done!')

In [ ]:
# # Reload model
# model_lgb = lgb.Booster(model_file=root_path + 'data/output/model/lgb-0.981609(r2100).txt')

In [ ]:
########### Model save and reload - end ###########